# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [1]:
# Install required packages
%pip install jupysql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db
#%config SqlMagic.style = 'table'

Note: you may need to restart the kernel to use updated packages.


Connecting to 'sqlite:///sql-murder-mystery.db'

In [2]:
%%sql
SELECT name
FROM sqlite_master

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
sqlite_autoindex_get_fit_now_member_1
get_fit_now_check_in
solution
check_solution
income



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [5]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [6]:
%%sql
INSERT INTO solution VALUES (1, 'Insert the name of the person you found here');
SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
That's not the right person. Try again!


In [3]:
# SQL Murder Mystery

# Step 1: Find the crime scene report
print("Finding the crime scene report\n")

crime_scene_report = %sql SELECT * FROM crime_scene_report WHERE date = 20180115 AND type = 'murder' AND city = 'SQL City';

# Print the crime scene report description
print("Crime Scene Report:")
for row in crime_scene_report:
    print(row.description)

Finding the crime scene report



Running query in 'sqlite:///sql-murder-mystery.db'

Crime Scene Report:
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


In [4]:
# Step 2: Find the first witness
print("\nFinding the first witness\n")

witness1 = %sql SELECT id, name, address_number, address_street_name FROM person WHERE address_street_name = 'Northwestern Dr' ORDER BY address_number DESC LIMIT 1;

print("First Witness:")
print(f"ID: {witness1[0].id}, Name: {witness1[0].name}")

# Find the second witness 
print("\nFinding the second witness\n")

witness2 = %sql SELECT id, name, address_number, address_street_name FROM person WHERE name LIKE '%Annabel%' AND address_street_name = 'Franklin Ave';

print("Second Witness:")
print(f"ID: {witness2[0].id}, Name: {witness2[0].name}")



Finding the first witness



Running query in 'sqlite:///sql-murder-mystery.db'

First Witness:
ID: 14887, Name: Morty Schapiro

Finding the second witness



Running query in 'sqlite:///sql-murder-mystery.db'

Second Witness:
ID: 16371, Name: Annabel Miller


In [6]:
# Step 3: Get witness statements
print("\nGetting witness statements\n")

try:
    # Validate witness1 and witness2
    if len(witness1) == 0 or len(witness2) == 0:
        raise ValueError("Witness data is missing. Ensure Step 2 executed correctly.")

    # Extract witness IDs
    witness1_id = int(witness1[0].id)
    witness2_id = int(witness2[0].id)
    
    # Create a string with the IDs for the SQL query
    witness_ids = f"{witness1_id}, {witness2_id}"
    
    result = %sql SELECT p.name, i.transcript FROM interview i JOIN person p ON i.person_id = p.id WHERE p.id IN ({{witness1_id}}, {{witness2_id}})
    
    # Validate result
    if len(result) == 0:
        print("No witness statements found.")
    else:
        print("Witness Statements:")
        for row in result:
            print(f"Name: {row.name}, Transcript: {row.transcript}")

except Exception as e:
    print(f"An error occurred while retrieving witness statements: {e}")


Getting witness statements



Running query in 'sqlite:///sql-murder-mystery.db'

Witness Statements:
Name: Morty Schapiro, Transcript: I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
Name: Annabel Miller, Transcript: I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


In [8]:
# Step 4: Check gym members with ID starting with "48Z" and gold status
print("\nFinding gym members matching the description\n")

try:
    # Get gold members with ID starting with 48Z
    gym_members = %sql SELECT m.id, p.name, m.membership_status, p.id as person_id FROM get_fit_now_member m JOIN person p ON m.person_id = p.id WHERE m.id LIKE '48Z%' AND m.membership_status = 'gold';
    
    if len(gym_members) == 0:
        print("No gym members found with the specified criteria.")
    else:
        print("Suspects from Gym (Gold members with ID starting with 48Z):")
        for member in gym_members:
            print(f"{member.name}: {member.id}, Person ID: {member.person_id}")
    
    # Check gym check-ins for the day of the murder (Jan 15, 2018)
    print("\nChecking gym check-ins for the day of the murder (Jan 15, 2018)\n")
    
    checkins_murder_day = %sql SELECT c.*, m.person_id, p.name FROM get_fit_now_check_in c JOIN get_fit_now_member m ON c.membership_id = m.id JOIN person p ON m.person_id = p.id WHERE c.check_in_date = 20180115 AND m.id LIKE '48Z%';
    
    if len(checkins_murder_day) == 0:
        print("No check-ins found on the day of the murder for the suspects.")
    else:
        print("Gym Check-ins on the day of the murder:")
        for checkin in checkins_murder_day:
            print(f"{checkin.name}: {checkin.membership_id}, Check-in: {checkin.check_in_time}, Check-out: {checkin.check_out_time}")
    
    # Check gym check-ins for January 9th (from Annabel's statement)
    print("\nChecking gym check-ins for January 9th (mentioned by Annabel)\n")
    
    checkins_jan9 = %sql SELECT c.*, m.person_id, p.name FROM get_fit_now_check_in c JOIN get_fit_now_member m ON c.membership_id = m.id JOIN person p ON m.person_id = p.id WHERE c.check_in_date = 20180109 AND m.id LIKE '48Z%';
    
    if len(checkins_jan9) == 0:
        print("No check-ins found on January 9th for the suspects.")
    else:
        print("Gym Check-ins on January 9th:")
        for checkin in checkins_jan9:
            print(f"{checkin.name}: {checkin.membership_id}, Check-in: {checkin.check_in_time}, Check-out: {checkin.check_out_time}")

except Exception as e:
    print(f"An error occurred while checking gym information: {e}")



Finding gym members matching the description



Running query in 'sqlite:///sql-murder-mystery.db'

Suspects from Gym (Gold members with ID starting with 48Z):
Joe Germuska: 48Z7A, Person ID: 28819
Jeremy Bowers: 48Z55, Person ID: 67318

Checking gym check-ins for the day of the murder (Jan 15, 2018)



Running query in 'sqlite:///sql-murder-mystery.db'

No check-ins found on the day of the murder for the suspects.

Checking gym check-ins for January 9th (mentioned by Annabel)



Running query in 'sqlite:///sql-murder-mystery.db'

Gym Check-ins on January 9th:
Joe Germuska: 48Z7A, Check-in: 1600, Check-out: 1730
Jeremy Bowers: 48Z55, Check-in: 1530, Check-out: 1700


In [10]:
# Step 5: Prepare a list of person IDs from the gym members to check their car plates
print("\nChecking driver's license info for suspects\n")

try:
    # Verify gym_members exists and contains data
    if 'gym_members' not in locals() or len(gym_members) == 0:
        raise ValueError("Gym members data is missing. Please ensure Step 4 executed correctly.")
    
    # Create a list of person IDs as a string for the SQL query
    person_ids = ', '.join(str(member.person_id) for member in gym_members)
    
    # Log the person IDs
    print(f"Looking for license plates containing 'H42W' for person IDs: {person_ids}")
    
    # Execute the query 
    suspect_licenses = %sql SELECT p.name, p.id as person_id, dl.* FROM drivers_license dl JOIN person p ON dl.id = p.license_id WHERE p.id IN ({{person_ids}}) AND dl.plate_number LIKE '%H42W%';
    
    # Check if any suspects match the car description
    if len(suspect_licenses) == 0:
        print("No suspects found with license plates containing 'H42W'.")
    else:
        print("Suspects matching car description:")
        for suspect in suspect_licenses:
            print(f"Name: {suspect.name}")
            print(f"License ID: {suspect.id}")
            print(f"Car: {suspect.car_make} {suspect.car_model}, Plate: {suspect.plate_number}")

except Exception as e:
    print(f"An error occurred while checking driver's license info: {e}")



Checking driver's license info for suspects

Looking for license plates containing 'H42W' for person IDs: 28819, 67318


Running query in 'sqlite:///sql-murder-mystery.db'

Suspects matching car description:
Name: Jeremy Bowers
License ID: 423327
Car: Chevrolet Spark LS, Plate: 0H42W2


In [12]:
# Step 6: Find our primary suspect
print("\nIdentifying the primary suspect\n")

try:
    # Verify suspect_licenses exists and contains data
    if 'suspect_licenses' not in locals():
        raise ValueError("Suspect license data is missing. Please ensure Step 5 executed correctly.")
    
    if len(suspect_licenses) > 0:
        primary_suspect = suspect_licenses[0].name
        print(f"The primary suspect is: {primary_suspect}")
    else:
        print("No primary suspect identified with the given criteria.")
        primary_suspect = "Unknown"
    
    # Summarize evidence against the primary suspect
    if primary_suspect != "Unknown":
        print("\nEvidence summary:")
        print(f"1. {primary_suspect} is a gold member of Get Fit Now Gym with ID starting with '48Z'")
        print(f"2. {primary_suspect} was at the gym on January 9th when Annabel saw the killer")
        print(f"3. {primary_suspect} owns a car with license plate containing 'H42W'")
        print("\nAll evidence from witness statements points to this suspect.")

except Exception as e:
    print(f"An error occurred while identifying the primary suspect: {e}")
    primary_suspect = "Unknown"



Identifying the primary suspect

The primary suspect is: Jeremy Bowers

Evidence summary:
1. Jeremy Bowers is a gold member of Get Fit Now Gym with ID starting with '48Z'
2. Jeremy Bowers was at the gym on January 9th when Annabel saw the killer
3. Jeremy Bowers owns a car with license plate containing 'H42W'

All evidence from witness statements points to this suspect.


In [14]:
# Step 7: Verify our solution
print("\nVerifying our solution\n")

try:
    # Verify primary_suspect exists and contains data
    if 'primary_suspect' not in locals() or primary_suspect is None:
        raise ValueError("Primary suspect data is missing. Please ensure Step 6 executed correctly.")
    
    %sql DELETE FROM solution;
    
    # Insert the primary suspect's name and check the solution
    solution_check = %sql INSERT INTO solution VALUES (1, '{{primary_suspect}}'); SELECT value FROM solution;
    
    # Check if we got a result
    if len(solution_check) > 0:
        print(f"Solution check: {solution_check[0].value}")
        
        # Evaluate the result
        if "Congrats" in str(solution_check[0].value):
            print("\nCongratulations! You've solved the murder mystery!")
        else:
            print("\nHmm, that doesn't seem to be the right suspect. Let's double-check our evidence.")
    else:
        print("No solution verification result returned.")

except Exception as e:
    print(f"An error occurred while verifying the solution: {e}")


Verifying our solution



Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

Solution check: Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

Congratulations! You've solved the murder mystery!


In [21]:
# Step 8: Extract Jeremy Bowers' confession
print("\nExtracting Jeremy Bowers' confession\n")

try:
    # Query Jeremy Bowers' interview transcript
    jeremy_confession = %sql SELECT transcript FROM interview WHERE person_id = (SELECT id FROM person WHERE name = 'Jeremy Bowers');
    
    if len(jeremy_confession) == 0:
        print("No confession found for Jeremy Bowers.")
    else:
        confession_text = jeremy_confession[0].transcript
        print("Jeremy Bowers' confession:")
        print(confession_text)
        
except Exception as e:
    print(f"An error occurred while retrieving Jeremy's confession: {e}")


Extracting Jeremy Bowers' confession



Running query in 'sqlite:///sql-murder-mystery.db'

Jeremy Bowers' confession:
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



In [28]:
# Step 9: Find the mastermind based on Jeremy's confession
print("\nFinding the mastermind based on Jeremy's confession\n")

try:
    # Based on Jeremy's actual confession, we know the mastermind has these characteristics:
    # - Woman with red hair
    # - Height between 65-67 inches (5'5" to 5'7")
    # - Drives a Tesla Model S
    # - Attended the SQL Symphony Concert 3 times in December 2017
    
    print("Executing query to find the mastermind...\n")
    
    # The gender information is likely stored in the drivers_license table as well
    real_mastermind = %sql SELECT p.name FROM person p JOIN drivers_license dl ON p.license_id = dl.id WHERE dl.hair_color = 'red' AND dl.car_make = 'Tesla' AND dl.car_model = 'Model S' AND dl.height >= 65 AND dl.height <= 67 AND dl.gender = 'female' AND p.id IN (SELECT f.person_id FROM facebook_event_checkin f WHERE f.event_name = 'SQL Symphony Concert' AND f.date LIKE '201712%' GROUP BY f.person_id HAVING COUNT(*) = 3)
    
    if len(real_mastermind) > 0:
        mastermind_name = real_mastermind[0].name
        print(f"The mastermind behind the murder is: {mastermind_name}")
        
        # Verify the solution
        print("\nVerifying our final solution...\n")
        %sql DELETE FROM solution;
        final_solution = %sql INSERT INTO solution VALUES (1, '{{mastermind_name}}'); SELECT value FROM solution;
        
        print(f"Final solution check: {final_solution[0].value}")
        
        if "Congrats" in str(final_solution[0].value):
            print("\nExcellent work, detective! You've uncovered the true mastermind behind the crime!")
        else:
            print("\nThat doesn't seem to be the real mastermind. Let's review the evidence again.")
    else:
        print("No mastermind found matching the criteria from Jeremy's confession.")

except Exception as e:
    print(f"An error occurred while finding the mastermind: {e}")


Finding the mastermind based on Jeremy's confession

Executing query to find the mastermind...



Running query in 'sqlite:///sql-murder-mystery.db'

The mastermind behind the murder is: Miranda Priestly

Verifying our final solution...



Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

Final solution check: Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!

Excellent work, detective! You've uncovered the true mastermind behind the crime!
